In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import pandas as pd
import os
import datetime
import logging
logging.basicConfig(filename="error.log", level=logging.INFO)

In [ ]:
home_url = 'https://www.espncricinfo.com/records/list-of-match-results-by-year-307852'

home_urlclient = urlopen(home_url)

home_page = home_urlclient.read()

home_page_html = bs(home_page, 'html.parser')

In [ ]:
home_bigbox = home_page_html.findAll(
    "a", {"class": "ds-inline-flex ds-items-start ds-leading-none"})
del home_bigbox[-5:]

In [ ]:
for match in home_bigbox:
    try:
        try:
            os.mkdir(match.text)
            logging.info('Directory creation successful')
        except OSError:
            logging.info('Directory creation failed')

        url = 'https://www.espncricinfo.com'+match['href']

        urlclient = urlopen(url)

        page = urlclient.read()

        page_html = bs(page, 'html.parser')

        df = pd.read_html(page)

        # Dataframe containing all the match details
        df = df[0]
        bigbox = page_html.findAll(
            "td", {"class": "ds-min-w-max ds-text-right"})

        # Cleaning elements
        del bigbox[0:9]

        # Extracting all matches details link
        matchlink = []
        i = int(0)
        while i < (int(len(bigbox))):
            matchlink.append('https://www.espncricinfo.com' +
                             (bigbox[i].a['href']))
            i = i + 5
            
        df.rename(columns={'Scorecard': 'match_id'},inplace=True)  

        # Saving the dataset
        df.to_csv(match.text+"/Summary_T20_"+match.text+".csv", index=False)

        # Nested Scraping inside url
        
        # Scraping Batting Data
        df_batting = pd.DataFrame(columns=['match', 'teamInnings', 'battingPos',
                                           'batsmanName', 'runs', 'balls', '4s', '6s', 'SR', 'out/not_out', 'match_id'])

        count = int(0)
        # for i in range(5):
        for i in range(len(matchlink)):
            try:
                urln = matchlink[i]
                df0 = df['match_id'][i]
                urlclient = urlopen(urln)
                page = urlclient.read()
                page_htmln = bs(page, 'html.parser')

                # 1st Innings
                page_html_bat_in1 = page_htmln.findAll(
                    "table", {"class": "ds-w-full"})[0].findAll("td", {"class": "ds-items-center"})
                del page_html_bat_in1[-1]
                for j in range(len(page_html_bat_in1)):

                    # battingPos
                    df_batting.loc[count, 'battingPos'] = j+1

                    # batsmanName
                    df_batting.loc[count,
                                   'batsmanName'] = page_html_bat_in1[j].a['title']

                    # match
                    match_countries = page_htmln.findAll("span", {
                        "class": "ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate"})
                    match_str = match_countries[0].text + \
                        (" Vs ") + match_countries[1].text
                    df_batting.loc[count, 'match'] = (match_str)

                    # teamInnings
                    df_batting.loc[count, 'teamInnings'] = (page_htmln.find(
                        "span", {"class": "ds-text-title-xs ds-font-bold ds-capitalize"}).text)

                    # match_id
                    df_batting.loc[count, 'match_id'] = df['match_id'][i]

                    # runs
                    df_batting.loc[count, 'runs'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[6*j].text)

                    # balls
                    df_batting.loc[count, 'balls'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[6*j+1].text)

                    # 4s
                    df_batting.loc[count, '4s'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[6*j+3].text)

                    # 6s
                    df_batting.loc[count, '6s'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[6*j+4].text)

                    # SR
                    df_batting.loc[count, 'SR'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[6*j+5].text)

                    # out/not_out
                    ot = page_htmln.findAll("table", {"class": "ds-w-full"})[0].findAll(
                        "td", {"class": "ds-min-w-max !ds-pl-[100px]"})[j].text
                    if (ot == 'not out '):
                        df_batting.loc[count, 'out/not_out'] = 'not_out'
                    else:
                        df_batting.loc[count, 'out/not_out'] = 'out'

                    count += 1

                # 2nd Inning
                page_html_bat_in2 = page_htmln.findAll(
                    "table", {"class": "ds-w-full"})[2].findAll("td", {"class": "ds-items-center"})
                del page_html_bat_in2[-1]
                for j in range(len(page_html_bat_in2)):

                    # battingPos
                    df_batting.loc[count, 'battingPos'] = j+1

                    # batsmanName
                    df_batting.loc[count,
                                   'batsmanName'] = page_html_bat_in2[j].a['title']

                    # match
                    match_countries = page_htmln.findAll("span", {
                        "class": "ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate"})
                    match_str = match_countries[0].text + \
                        (" Vs ") + match_countries[1].text
                    df_batting.loc[count, 'match'] = (match_str)

                    # teamInnings
                    df_batting.loc[count, 'teamInnings'] = page_htmln.findAll(
                        "span", {"class": "ds-text-title-xs ds-font-bold ds-capitalize"})[1].text

                    # match_id
                    df_batting.loc[count, 'match_id'] = df['match_id'][i]

                    lp = len(page_html_bat_in1)
                    lb = len(page_htmln.findAll(
                        "table", {"class": "ds-w-full"})[1].findAll("td", {"ds-flex ds-items-center"}))
                    l = 6*lp+9*lb
                    # 6 * no of plyers in last innings(lp)
                    # 9 * no of ballers in last innings(lb)

                    # runs
                    df_batting.loc[count, 'runs'] = page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[l+6*j].text

                    # balls
                    df_batting.loc[count, 'balls'] = page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[1+l+6*j].text

                    # 4s
                    df_batting.loc[count, '4s'] = page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[3+l+6*j].text

                    # 6s
                    df_batting.loc[count, '6s'] = page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[4+l+6*j].text

                    # SR
                    df_batting.loc[count, 'SR'] = page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[5+l+6*j].text

                    # out/not_out
                    ot = page_htmln.findAll("table", {"class": "ds-w-full"})[2].findAll(
                        "td", {"class": "ds-min-w-max !ds-pl-[100px]"})[j].text
                    if (ot == 'not out '):
                        df_batting.loc[count, 'out/not_out'] = 'not_out'
                    else:
                        df_batting.loc[count, 'out/not_out'] = 'out'

                    count += 1

            except Exception as e:
                logging.info(e)

        # Saving the dataset
        df_batting.to_csv(
            match.text+"/Batting_PerMatchData_"+match.text+".csv", index=False)

        # Scraping Bowling Data
        df_bowling = pd.DataFrame(columns=['match', 'bowlingTeam', 'bowlerName', 'overs',
                                           'maiden', 'runs', 'wickets', 'economy', '0s', '4s', '6s', 'wides', 'noBalls', 'match_id'])

        count = int(0)
        # for i in range(5):
        for i in range(len(matchlink)):
            try:
                urln = matchlink[i]
                df0 = df['match_id'][i]
                urlclient = urlopen(urln)
                page = urlclient.read()
                page_htmln = bs(page, 'html.parser')

                # 1st Innings
                page_html_bat_in1 = page_htmln.findAll(
                    "table", {"class": "ds-w-full"})[0].findAll("td", {"class": "ds-items-center"})
                page_html_bowl_in1 = page_htmln.findAll(
                    "table", {"class": "ds-w-full"})[1].findAll("td", {"class": "ds-items-center"})
                del page_html_bat_in1[-1]
                for j in range(len(page_html_bowl_in1)):

                    df_bowling.loc[count,
                                   'bowlerName'] = page_html_bowl_in1[j].text

                    # match
                    match_countries = page_htmln.findAll("span", {
                        "class": "ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate"})
                    match_str = match_countries[0].text + \
                        (" Vs ") + match_countries[1].text
                    df_bowling.loc[count, 'match'] = (match_str)

                    # bowlingTeam
                    df_bowling.loc[count,
                                   'bowlingTeam'] = match_countries[1].text

                    # match_id
                    df_bowling.loc[count, 'match_id'] = df['match_id'][i]

                    lp = 6 * len(page_html_bat_in1)
                    # 6 * no of plyers in last innings(lp)

                    # overs
                    df_bowling.loc[count, 'overs'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[lp + 9*j].text)

                    # maiden
                    df_bowling.loc[count, 'maiden'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[lp + 9*j+1].text)

                    # runs
                    df_bowling.loc[count, 'runs'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[lp + 9*j+2].text)

                    # economy
                    df_bowling.loc[count, 'economy'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[lp + 9*j+3].text)

                    # 0s
                    df_bowling.loc[count, '0s'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[lp + 9*j+4].text)

                    # 4s
                    df_bowling.loc[count, '4s'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[lp + 9*j+5].text)

                    # 6s
                    df_bowling.loc[count, '6s'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[lp + 9*j+6].text)

                    # wides
                    df_bowling.loc[count, 'wides'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[lp + 9*j+7].text)

                    # noBalls
                    df_bowling.loc[count, 'noBalls'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[lp + 9*j+8].text)

                    # wickets
                    df_bowling.loc[count, 'wickets'] = (page_htmln.findAll("table", {"class": "ds-w-full ds-table ds-table-md ds-table-auto"})[
                                                        0].findAll("td", {"class": "ds-w-0 ds-whitespace-nowrap ds-text-right"}))[j].text

                    count += 1

                # 2nd Inning
                page_html_bat_in2 = page_htmln.findAll(
                    "table", {"class": "ds-w-full"})[2].findAll("td", {"class": "ds-items-center"})
                del page_html_bat_in2[-1]
                page_html_bowl_in2 = page_htmln.findAll(
                    "table", {"class": "ds-w-full"})[3].findAll("td", {"class": "ds-items-center"})
                # del page_html_bowl_in2[-1]
                for j in range(len(page_html_bowl_in2)):

                    # bowlerName
                    df_bowling.loc[count,
                                   'bowlerName'] = page_html_bowl_in2[j].text

                    # match
                    match_countries = page_htmln.findAll("span", {
                        "class": "ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate"})
                    match_str = match_countries[0].text + \
                        (" Vs ") + match_countries[1].text
                    df_bowling.loc[count, 'match'] = (match_str)

                    # bowlingTeam
                    df_bowling.loc[count,
                                   'bowlingTeam'] = match_countries[0].text

                    # match_id
                    df_bowling.loc[count, 'match_id'] = df['match_id'][i]

                    lp2 = 6 * len(page_html_bat_in2)
                    lb = 9*len(page_html_bowl_in1)
                    l = lp + lb + lp2
                    # 9 * no of ballers in last innings(lb)

                    # overs
                    df_bowling.loc[count, 'overs'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[l+9*j].text)

                    # maiden
                    df_bowling.loc[count, 'maiden'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[l + 9*j+1].text)

                    # runs
                    df_bowling.loc[count, 'runs'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[l + 9*j+2].text)

                    # economy
                    df_bowling.loc[count, 'economy'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[l + 9*j+3].text)

                    # 0s
                    df_bowling.loc[count, '0s'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[l + 9*j+4].text)

                    # 4s
                    df_bowling.loc[count, '4s'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[l + 9*j+5].text)

                    # 6s
                    df_bowling.loc[count, '6s'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[l + 9*j+6].text)

                    # wides
                    df_bowling.loc[count, 'wides'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[l + 9*j+7].text)

                    # noBalls
                    df_bowling.loc[count, 'noBalls'] = (page_htmln.findAll(
                        "td", {"class": "ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right"})[l + 9*j+8].text)

                    # wickets
                    df_bowling.loc[count, 'wickets'] = (page_htmln.findAll("table", {"class": "ds-w-full ds-table ds-table-md ds-table-auto"})[
                                                        1].findAll("td", {"class": "ds-w-0 ds-whitespace-nowrap ds-text-right"}))[j].text

                    count += 1

            except Exception as e:
                logging.info(e)

        # Saving the bowling dataset
        df_bowling.to_csv(
            match.text+"/Bowling_PerMatchData_"+match.text+".csv", index=False)

    except Exception as e:
        logging.info(e)

In [ ]:
ls = os.listdir()
directories = [directory for directory in ls if os.path.isdir(
    directory) and directory.startswith("20")]

In [ ]:
Combined_PerMatch_Batting_Data_T20 = pd.DataFrame(columns=['match', 'teamInnings', 'battingPos',
                                                           'batsmanName', 'runs', 'balls', '4s', '6s', 'SR', 'out/not_out', 'match_id'])

Combined_PerMatch_Bowling_Data_T20 = pd.DataFrame(columns=['match', 'bowlingTeam', 'bowlerName', 'overs',
                                                           'maiden', 'runs', 'wickets', 'economy', '0s', '4s', '6s', 'wides', 'noBalls', 'match_id'])


Combined_Summary_T20 = pd.DataFrame(
    columns=['Team 1', 'Team 2', 'Winner', 'Margin', 'Ground', 'Match Date', 'match_id'])

for dir in directories:
    # Combining batting data
    df_bat = pd.read_csv(dir+("/")+"Batting_PerMatchData_"+dir+".csv")
    Combined_PerMatch_Batting_Data_T20 = pd.concat(
        [Combined_PerMatch_Batting_Data_T20, df_bat], axis=0)

    # Combining bowling data
    df_bowl = pd.read_csv(dir+("/")+"Bowling_PerMatchData_"+dir+".csv")
    Combined_PerMatch_Bowling_Data_T20 = pd.concat(
        [Combined_PerMatch_Bowling_Data_T20, df_bowl], axis=0)

    # Combining Summary
    df_summary = pd.read_csv(dir+("/")+"Summary_T20_"+dir+".csv")
    Combined_Summary_T20 = pd.concat(
        [Combined_Summary_T20, df_summary], axis=0)


date = datetime.date.today().strftime("%B_%d_%Y")

Combined_PerMatch_Batting_Data_T20.to_csv(
    "Combined_Batting_PerMatchData_T20_2005_to_"+date+".csv", index=False)
Combined_PerMatch_Bowling_Data_T20.to_csv(
    "Combined_Bowling_PerMatchData_T20_2005_to_"+date+".csv", index=False)
Combined_Summary_T20.to_csv("Combined_Summary_T20_2005_to_"+date+".csv", index=False)